 
# Apresentação do cenário de negócios

Você trabalha para um provedor de serviços médicos e deseja melhorar a detecção de anormalidades em pacientes ortopédicos. 

Você tem a incumbência de resolver esse problema usando machine learning (ML). Você tem acesso a um conjunto de dados que contém seis componentes biomecânicos (features) e um alvo (target) de *normal* (normal) ou *abnormal* (anormal). Você pode usar esse conjunto de dados (datasets) para treinar um modelo de ML para prever se um paciente terá uma anomalia.

## Sobre esse conjunto de dados
Esse conjunto de dados (dataset) biomédicos foi criado pelo Dr. Henrique da Mota durante um período de residência médica no Group of Applied Research in Orthopaedics (GARO) do Centre Médico-Chirurgical de Réadaptation des Massues em Lyon, na França. Os dados foram organizados em duas tarefas de classificação diferentes, mas relacionadas. 

A primeira tarefa consiste em classificar os pacientes como pertencentes a uma das três categorias a seguir: 

- *Normal* (Normal) (100 pacientes)
- *Disk Hernia* (Hérnia de disco) (60 pacientes)
- *Spondylolisthesis* (Espondilolistese) (150 pacientes)

As categorias *Disk Hernia* (Hérnia de disco) e *Spondylolisthesis* (Espondilolistese) foram mescladas em uma única categoria, rotulada como *abnormal* (anormal). Portanto, a atividade consiste em classificar os pacientes como pertencentes a uma das categorias: *Normal* (Normal) (100 pacientes) ou *Abnormal* (Anormal) (210 pacientes).


## Informações de atributo

Cada paciente é representado no conjunto de dados por seis atributos biomecânicos derivados da forma e da orientação da pelve e da coluna lombar (nesta ordem): 

- Incidência pélvica
- Inclinação pélvica
- Ângulo da lordose lombar
- Inclinação sacral
- Raio pélvico
- Grau de espondilolistese

A convenção a seguir é usada para os rótulos de classe (labels): 
- DH (hérnia de disco)
- Espondilolistese (SL)
- Normal (NO) 
- Anormal (AB)


Para obter mais informações sobre esse conjunto de dados, consulte a [página da Web Conjunto de dados de coluna vertebral](http://archive.ics.uci.edu/ml/datasets/Vertebral+Column).



# Coletar e Rotular o Conjunto de Dados (dataset)

## Fonte dos dados

Dua, D. and Graff, C. (2019). UCI machine learning Repository: https://archive.ics.uci.edu/ml/index.php. Irvine, CA: University of California, School of Information and Computer Science.

In [1]:
url_dataset = 'http://archive.ics.uci.edu/ml/machine-learning-databases/00212/vertebral_column_data.zip'

### Desafios:

1. Os dados tem que ser acessados via web.
2. Os dados estão compactados (zipfiles).
3. Os dados estão no formato arff (comum em arquivos usados em aprendizado de máquina)

### Pacote requests  

O objetivo da pacote __requests__ é tornar as solicitações HTTP mais simples e mais amigáveis.

In [14]:
# Exemplo de uso do pacote requests
import requests
r = requests.get('https://www.impacta.com.br')

print(r.status_code)

200


Primeiro vamos baixar os dados que estão em um servidor. A próxima célula faz uma solicitação de dados a um  servidor escolhido.

In [16]:
url_dataset = 'http://archive.ics.uci.edu/ml/machine-learning-databases/00212/vertebral_column_data.zip'

# copia os dados do sevido
r = requests.get(url_dataset, stream=True)

Nesta célula, a função **requests.get()** é usada para enviar uma solicitação HTTP GET para a URL **url_dataset**. 

A variável **r** conterá o objeto de resposta retornado por request.get(). Ao definir **stream=True**, o conteúdo da resposta será baixado em blocos, em vez de carregar a resposta inteira na memória de uma vez. Isso é útil para lidar com arquivos grandes ou casos em que você deseja processar os dados de forma incremental.

O objeto **r** pode ser processado posteriormente para acessar o conteúdo baixado ou salvá-lo em um arquivo, dependendo do noso caso de uso específico.

### Pacote zipfile e io

Como os dados recebidos estão compactados, usaremos o pacote zipfile para manipular esses dados.

In [17]:
import zipfile

Como construtor **zipfile.ZipFile** espera um objeto semelhante a um arquivo como entrada, vamos usar os recursos  biblioteca **io** para transformar os dados mantidos em memória na variável **r**, em um estrutura de arquivo.

In [18]:
import io # O pacote IO serve para lidar com vários tipos de E/S.

Em seguida, o stream de dados recebidos é passado para a função ZipFile para extrair os dados.

In [19]:
# Extração do arquivo .zip
vertebral_zip = zipfile.ZipFile(io.BytesIO(r.content))
vertebral_zip.extractall()

Ao usar **io.BytesIO(r.content)**, estamos criando um objeto semelhante a um arquivo que pode ser passado para o construtor **ZipFile**.

O uso de **io.BytesIO** nos permite trabalhar com o conteúdo do arquivo sem precisar salvá-lo no disco. Ele fornece uma maneira conveniente de lidar com dados na memória e permite que executemos operações como extrair o conteúdo de um arquivo .zip diretamente dos bytes.

No visualizador de arquivos no painel de navegação à esquerda, você verá quatro arquivos novos:
- column_2C_weka.arff
- column_2C.dat
- column_3C_weka.arff 
- column_3C.dat


### Pacote Scipy

Como os dados recebidos estão no formato **ARRF**, vamos primeiro eliminar os metadados antes de começar a analisarmos os dados propriamente dito. Para essa tarefa usaremos o pacote **scipy**.

In [20]:
from scipy.io import arff

ARFF (Attribute-Relation File Format) é um formato de arquivo utilizado para representar dados estruturados em um formato tabular. 
Os arquivos ARFF são semelhantes aos arquivos CSV (Comma-Separated Values), pois também armazenam dados em formato tabular. No entanto, os arquivos ARFF possuem recursos adicionais para especificar informações sobre os atributos e seus tipos.

Um arquivo ARFF consiste em duas seções principais: a seção de cabeçalho e a seção de dados.
A seção de cabeçalho contém informações sobre os atributos presentes nos dados, incluindo seus nomes, tipos (numérico, nominal, etc.) e possíveis valores. Essas informações ajudam a definir a estrutura dos dados e permitem uma interpretação adequada.

A seção de dados contém os próprios dados organizados em linhas e colunas, seguindo a estrutura definida na seção de cabeçalho. Cada linha representa uma instância ou exemplo, enquanto as colunas correspondem aos atributos.

Os arquivos ARFF são comumente utilizados em conjunto com ferramentas de mineração de dados e aprendizado de máquina, como o Weka, para importar, exportar e analisar conjuntos de dados. Eles fornecem uma estrutura padronizada e facilmente interpretável para compartilhar e trabalhar com dados tabulares.

In [21]:
# Le o arquivo column_2C_weka.arff 
dados_metadados = arff.loadarff('column_2C_weka.arff')

A função **loadarff** é chamada com o nome de arquivo '**column_2C_weka.arff'** como parâmetro de entrada. A função lê o arquivo ARFF e retorna uma tupla contendo dois elementos: os dados e os metadados.

A variável dados é atribuída a esta tupla. Normalmente, **dados_metadados[0]** conterá os dados reais, que podem ser acessados para processamento ou análise posterior. Os metadados, armazenados em **dados_metadados[1]**, fornecem informações sobre os atributos, seus tipos e qualquer informação adicional especificada no arquivo ARFF.

Para acessar os dados, podemos usar **dados_metadados[0]** ou atribuí-los a uma variável separada, como **dados = dados_metadados[0]**. A partir daí, podemos manipular e analisar os dados conforme necessário.

In [23]:
dados = dados_metadados[0]

### Usando Pandas para se familiarizar com seus dados
A primeira etapa em qualquer projeto de aprendizado de máquina é familiarizar-se com os dados. Usaremos a biblioteca Pandas para isso. O Pandas é a principal ferramenta que os cientistas de dados usam para explorar e manipular dados. A maioria das pessoas abrevia pandas em seu código como pd. Fazemos isso com o comando

In [24]:
import pandas as pd

A parte mais importante da biblioteca Pandas é o DataFrame. Um DataFrame contém o tipo de dados que você pode imaginar como uma tabela. Isso é semelhante a uma planilha no Excel ou a uma tabela em um banco de dados SQL.

O Pandas possui métodos poderosos para a maioria dos procedimentos desejamos fazer com esse tipo de dados.

In [25]:
# Coloca os dados lidos em um DataFrame e mostra na forma de uma tabela.
df = pd.DataFrame(dados)
df.head()

,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
0,63.027817,22.552586,39.609117,40.475232,98.672917,-0.254400,b'Abnormal'
1,39.056951,10.060991,25.015378,28.995960,114.405425,4.564259,b'Abnormal'
2,68.832021,22.218482,50.092194,46.613539,105.985135,-3.530317,b'Abnormal'
3,69.297008,24.652878,44.311238,44.644130,101.868495,11.211523,b'Abnormal'
4,49.712859,9.652075,28.317406,40.060784,108.168725,7.918501,b'Abnormal'


# NumPy

O __NumPy__ é o pacote fundamental para computação científica com Python. Entre outras coisas, ele contém:

- Objeto poderoso de matriz N-dimensional
- Funções sofisticadas (transmissão)
- Ferramentas para integrar código C/C++ e Fortran
- Recursos úteis de álgebra linear, transformação de Fourier e número aleatório

Podemos instalar o NumPy usandoo gerenciador de pacotes __*pip*__.

In [34]:
!pip install --upgrade pip
!pip install numpy

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 22.3.1
    Uninstalling pip-22.3.1:
      Successfully uninstalled pip-22.3.1
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


Para começar, primeiro importe o Numpy usando a instrução de importação do Python. Nomeie o Numpy como *np*, portanto, não precisamos escrever numpy cada vez que usarmos uma de suas funções. Esta é a convenção geral seguida ao importar a biblioteca.

In [26]:
import numpy as np

## Array Numpy

Um Array Numpy é muito semelhante a uma lista Python com uma condição especial de que todos os seus elementos devem ser do mesmo tipo. 

Usamos a função array() para definir um array em numpy. Ele aceita o primeiro argumento como a matriz e o segundo argumento como o tipo de elemento, por exemplo int, float etc. No exemplo, definimos uma matriz com os elementos 1, 2, 3 e 4 do tipo int (inteiro).

In [27]:
# Define um array com elementos do tipo inteiro: 1,2,3 e 4
arrayInteiros = np.array([1,2,3,4], int)

No NumPy, você pode utilizar diferentes tipos de dados para criar arrays. Além do tipo de dado 'int' (inteiro), você pode utilizar os seguintes tipos de dados para criar arrays:

- 'bool': para representar valores booleanos (True ou False).
- 'float': para representar números de ponto flutuante.
- 'complex': para representar números complexos.
- 'str': para representar strings (cadeias de caracteres).
- 'object': para representar objetos genéricos em Python.
- 'unicode': para representar strings Unicode.

Além desses tipos de dados básicos, o NumPy também fornece tipos de dados específicos com tamanhos fixos, como:

- 'int8', 'int16', 'int32', 'int64': para representar inteiros com diferentes tamanhos em bits.
- 'uint8', 'uint16', 'uint32', 'uint64': para representar inteiros sem sinal (não negativos) com diferentes tamanhos em bits.
- 'float16', 'float32', 'float64': para representar números de ponto flutuante com diferentes precisões.

Por exemplo, você pode criar um array de ponto flutuante da seguinte forma: **np.array([1.0, 2.5, 3.7], float)**'. Da mesma forma, você pode criar um array de strings utilizando '**np.array(["sim", "não", "talvez"], str)**'.

## Criação de um Array  (matriz)

O índice na matriz numpy também começa com 0, então **arrayInteiros[0]** se refere ao primeiro elemento que é 1. 

Também podemos definir um intervalo como [: 2] que imprime todos os valores nos índices 0 a 1.

É simples verificar se um elemento pertence a um array, basta utilizar o operador in, nesse caso a resposta é um valor booleano. 

In [37]:
# Podemos acessar esses elementos usando valores de índice
print(arrayInteiros[0])

# Também podemos usar intervalos para acessar valores
print(arrayInteiros[: 2])

# Descobre se existe um valor na matriz
# Retorna verdadeiro se o valor existir, caso contrário retorna falso
print(2 in arrayInteiros)

1
[1 2]
True


O Numpy também fornece métodos rápidos para definir uma matriz. Os métodos básicos incluem:

- **zeros**:  array com todos os valores zero
- **ones**: array com todos os valores um
- **arange**: define a matriz como um intervalo de valores
- **concatenação**: combinar matrizes

In [28]:
# Cria array
a = np.zeros(5) # Cria um array de zeros
b = np.ones((3,5)) # Cria um array de uns  
c = np.ones((2,3,4), dtype=np.int16 ) # Cria um array de uns do tipo inteiro

In [29]:
print(a)

[0. 0. 0. 0. 0.]


In [30]:
print(b)

[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]


In [31]:
print(c)

[[[1 1 1 1]
  [1 1 1 1]
  [1 1 1 1]]

 [[1 1 1 1]
  [1 1 1 1]
  [1 1 1 1]]]


In [32]:
# define uma array como um intervalo de valores
d = np.array(range(10), int) # Cria um array com valores entre 0 e 9
e = np.arange(15).reshape(3, 5) # Cria um array 3x5, com valores entre 0 e 14

In [33]:
print(d)

[0 1 2 3 4 5 6 7 8 9]


In [34]:
print(e)

[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]]


In [35]:
arrayInteiros2 = np.array([5,6], int)

# Concatenar dois arrays
f = np.concatenate((arrayInteiros, arrayInteiros2))

In [36]:
print(f)

[1 2 3 4 5 6]


## Números Aleatórios

Módulo random do Numpy é formado por um conjunto de funções baseadas na geração de números pseudoaleatórios. 
O número pseudoaleatório funciona começando com um inteiro chamado semente (seed) e então gera números em sucessão. A mesma semente fornece a mesma sequência de números aleatórios, daí o nome "pseudo" aleatório. 

In [37]:
x = np.random.random() # gera um valor entre 0 e 1.
print(x)

0.9105841250901242


In [38]:
x = np.random.randint(100) # gera um inteiro entre 0 e 100
print(x)

79


In [39]:
x = np.random.rand() # gera um ponto flutuante
print(x)

0.6402158275428212


In [40]:
x = np.random.random(10)
print(x)

[0.41150525 0.76881077 0.96888969 0.88985653 0.74361778 0.07897722
 0.00834204 0.75429794 0.73646596 0.56295257]


__Gerando um array com números aleatórios__

Os dois métodos já mostrados podem retornar matrizes com números aleatórios. 

In [41]:
g = np.random.randint(100, size=(6))
print(g)

[22 73 16 28 76 51]


In [42]:
h = np.random.randint(100, size=(15)).reshape(3, 5)
print(h)

[[13 92 72 45 32]
 [26 30 56 85 21]
 [83 44 68 91  9]]


In [43]:
i = np.random.rand(9).reshape(3, 3)
print(i)

[[0.80326016 0.70619581 0.19138176]
 [0.35027195 0.69969213 0.03652571]
 [0.1596831  0.02882841 0.77494813]]


__Valor Semente__

Conforme dito antes, os números pseudoaleatórios funcionam começando com um inteiro chamado semente e então gera números em sucessão. A mesma semente fornece a mesma sequência de números aleatórios, daí o nome "pseudo" geração de número aleatório. Se você deseja ter um código reproduzível, é bom propagar o gerador de números aleatórios usando a função __np.random.seed()__.

In [46]:
for j in range(5):
    i = np.random.rand(9).reshape(3, 3)
    print(i)

[[0.14371904 0.42858675 0.6315092 ]
 [0.64786408 0.19264569 0.0483815 ]
 [0.25706477 0.61370222 0.74051089]]
[[0.15225036 0.60673259 0.27955884]
 [0.92813404 0.26099805 0.92919857]
 [0.47811121 0.52625605 0.05364068]]
[[0.38722039 0.6667234  0.84619935]
 [0.1082074  0.78240306 0.94030343]
 [0.24912688 0.33551109 0.78354433]]
[[0.57447797 0.61607197 0.00635844]
 [0.9282422  0.78367007 0.09539526]
 [0.63886072 0.76024245 0.85642548]]
[[0.06362435 0.21393877 0.05252021]
 [0.13565937 0.72506674 0.29611614]
 [0.10909906 0.92766497 0.11758208]]


In [55]:
np.random.seed(170) # define o valor semente como sendo 17
for j in range(5):
    i = np.random.rand(9).reshape(3, 3)
    print(i)

[[0.05264542 0.22686178 0.27053051]
 [0.50443809 0.59693772 0.52525681]
 [0.74135356 0.01710367 0.94659158]]
[[0.46914048 0.6676281  0.34448804]
 [0.36016699 0.88204454 0.72518318]
 [0.55560337 0.02786747 0.27339284]]
[[0.71991029 0.90414274 0.5681797 ]
 [0.17786489 0.12039842 0.72450931]
 [0.24930328 0.68493792 0.76846214]]
[[0.60295516 0.90316022 0.17239175]
 [0.13017855 0.79891905 0.1845929 ]
 [0.71295853 0.95565049 0.83161521]]
[[0.47558039 0.98395327 0.9703749 ]
 [0.24367983 0.19250345 0.52824109]
 [0.08704815 0.45174819 0.0623505 ]]


## Inspecionando sua matriz

In [56]:
# Cria array
a = np.zeros(5) # Cria um array de zeros
b = np.ones((3,5)) # Cria um array de uns  
c = np.ones((2,3,4), dtype=np.int16 ) # Cria um array de uns do tipo inteiro
d = np.array(range(10), int) # Cria um array com valores entre 0 e 9

In [57]:
a.shape # Dimensões do array 

(5,)

In [58]:
len(b)# primeira dimensão do array

3

In [59]:
c.ndim # Número de dimensões do array

3

In [60]:
c.size # Número de elementos do array

24

In [61]:
b.dtype # Tipo dos dados

dtype('float64')

In [62]:
c.dtype.name # Nome do tipo de dados

'int16'

In [63]:
c.astype(float) # Converta um tipo de array em um tipo diferente

array([[[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]],

       [[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]]])

## Operações com os Arrays
Há uma ampla gama de operações que podemos aplicar em matrizes numpy.

### Operações matemáticas básicas

In [47]:
# Cria array
a = np.zeros(5) # Cria um array de zeros
b = np.ones((3,5)) # Cria um array de uns  
c = np.ones((2,3,4), dtype=np.int16 ) # Cria um array de uns do tipo inteiro
d = np.array([4,7,9,10,22]) 
e = np.array([2,7,26,5,5]) 

In [48]:
np.add(a,b) # Adição

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])

In [49]:
np.subtract(a,b) # Subtração

array([[-1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1.]])

In [50]:
np.divide(d,e) # Divisão

array([2.        , 1.        , 0.34615385, 2.        , 4.4       ])

In [51]:
np.multiply(d,e) # Multiplicação

array([  8,  49, 234,  50, 110])

In [52]:
np.array_equal(d,e) # Comparação

False

### Fuções sobre os elementos

In [53]:
# Cria os arrays
a = np.arange(15).reshape(3, 5) 

In [54]:
a.sum() # Soma os elementos

105

In [55]:
a.min() # Retorna o menor valor

0

In [56]:
a.mean() # Retorna o valor médio

7.0

In [57]:
np.median(a) # Retorna o valor mediana

7.0

In [58]:
a = np.arange(15).reshape(3, 5) 
a[1,2]=100
print(a)
a.max(axis=0) # valor máximo de cada coluna

[[  0   1   2   3   4]
 [  5   6 100   8   9]
 [ 10  11  12  13  14]]


array([ 10,  11, 100,  13,  14])

In [59]:
np.std(a) # Desvio Padrão

23.59717497215857

### Subconjunto, divisão e indexação

In [63]:
# Cria os arrays
a = np.arange(15)

In [64]:
a[0:2] # Selecione os itens no índice 0 e 1

array([0, 1])

In [65]:
a[:1] # Selecione todos os itens na linha 0

array([0])

In [66]:
a[-1:] # Selecione todos os itens da última linha

array([14])

In [67]:
a[a<2] # Selecione os elementos de 'a' que são menores que 2

array([0, 1])

In [63]:
# Cria os arrays
a = np.arange(15).reshape(3, 5) 

In [61]:
a[1,2] # Selecione o elemento da linha 1 e coluna 2

7